# MHCGlobe use cases
-----------

### 1. Predict peptide-MHC binding affinity

MHCGlobe predicts a binding score, and both the binding score and binding affinity (nM), `mhcglobe_score` and `mhcglobe_affinity`, respectively.

`MHCGlobe(train_type='full')` returns the MHCGlobe model trained on the full database of MHC-peptide binding instances in the present study, including both human and non-human pMHC data. 

In [1]:
import os
import pandas as pd
import sys
sys.path.append('./mhcglobe/')
sys.path.append('./mhcperf')
from mhcglobe import MHCGlobe

# Load MHCGlobe class object containing the fully trained model.
mhcglobe = MHCGlobe(train_type='full')

# Load binding data as CSV. Required columns are `allele` and `peptide`.
example_binding_data = './example/example_binding_data.csv'
prediction_cols = ['allele', 'peptide']
pMHC_data = pd.read_csv(example_binding_data, usecols=prediction_cols)

# Predict peptide-MHC binding with MHCGlobe
mhcglobe.predict_on_dataframe(pMHC_data)

,allele,peptide,mhcglobe_affinity,mhcglobe_score
0,HLA-B*15:01,AQMWSLMYF,64.439289,0.614990
1,HLA-A*33:01,DIDILQTNSR,396.716782,0.447011
2,HLA-B*38:01,QPKKAAAAL,20009.974714,0.084641
3,HLA-A*31:01,FLALGFFLR,117.473925,0.559490
4,HLA-B*07:02,RPQKRPSCI,104.042288,0.570712
...,...,...,...,...
95,HLA-A*30:01,KAFNHASVK,118.069387,0.559023
96,HLA-A*02:01,WMMAMKYPI,61.744092,0.618939
97,HLA-A*02:03,FLMGFNRDV,30.050617,0.685494
98,HLA-A*02:01,LLSTTEWQV,91.183992,0.582905


### 2. Re-train MHCGlobe

MHCGlobe contains an ensemble of deep neural network models, which can easily trained on user defined peptide-MHC binding data. We recomend training MHCGlobe using the initialized weights and biases that were used in the paper for comparable results. The recomended initalized MHCGlobe model can be accessed using `MHCGlobe(train_type='init')`.


In [3]:
# Load example training data.
training_cols = ['allele', 'peptide', 'measurement_value', 'measurement_inequality']
pMHC_data = pd.read_csv(example_binding_data, usecols=training_cols)

# Re-train
new_mhcglobe_path = '/tf/local/mhcglobe_models/mhcglobe_exp1'
new_mhcglobe = MHCGlobe('init').train_ensemble(
        df_train          = pMHC_data,
        new_mhcglobe_path = new_mhcglobe_path,
        verbose           = 0)

Training...
Training complete.


In [5]:
# Load user trained MHCGlobe
new_mhcglobe = MHCGlobe(new_mhcglobe_path=new_mhcglobe_path)

# Predict peptide-MHC binding with MHCGlobe re-trained by user.
new_mhcglobe.predict_on_dataframe(pMHC_data.loc[:, prediction_cols])

,allele,peptide,mhcglobe_affinity,mhcglobe_score
0,HLA-B*15:01,AQMWSLMYF,7.328021,0.815920
1,HLA-A*33:01,DIDILQTNSR,62.909557,0.617210
2,HLA-B*38:01,QPKKAAAAL,733.625455,0.390191
3,HLA-A*31:01,FLALGFFLR,15.014027,0.749627
4,HLA-B*07:02,RPQKRPSCI,31.224701,0.681952
...,...,...,...,...
95,HLA-A*30:01,KAFNHASVK,15.070312,0.749281
96,HLA-A*02:01,WMMAMKYPI,15.771884,0.745075
97,HLA-A*02:03,FLMGFNRDV,4.789568,0.855224
98,HLA-A*02:01,LLSTTEWQV,24.788067,0.703288


# MHCPerf use cases
__________

The fully trained MHCPerf will be used to predict allele-level performance of a given MHCGlobe model based on its training data featurized.

### 1. Estimate allele-level PPV for query MHC alleles given a pMHC binding dataset.

MHCPerf features will be created for all query alleles based on this data count dictionary, relating the binding dataset to each of the query alleles.

In [6]:
#sys.path.append('./mhcperf_BEST/') # make sure mhcglobe doesn't get trained with mhcperf scripts.

import mhcperf

# Load MHCPerf model
mhcperf_model = mhcperf.model()

# Compute MHCPerf features for query alleles.
query_alleles = ['HLA-B*15:13','HLA-A*30:03','HLA-C*14:02']
mhcglobe_training_data = '/tf/natmtd/data/mhcglobe_data/mhcglobe_full_train_data.csv'
perf_features = mhcperf.featurize_from_binding(mhcglobe_training_data, query_alleles)

# Predict PPV for each MHC allele in `query_alleles`
query_allele_ppv_est = mhcperf_model.predict_ppv(perf_features)

# Show MHCPerf allele-level PPV estimates.
pd.DataFrame(zip(query_alleles, query_allele_ppv_est), columns=['allele', 'PPV_est'])

,allele,PPV_est
0,HLA-B*15:13,0.761318
1,HLA-A*30:03,0.563628
2,HLA-C*14:02,0.629275


### 2. MHCPerf Hyperparameter Tuning and Training

MHCPerf is a single neural network which can be retained on new observations of MHCGlobe PPV performance in response to changes to the MHCGlobe training set. MHCPerf re-training repeats hyperparameter optimization using the grid search algorithm. This procedure can be run in the notebook (as shown below) or run in parallel in the backgroun using the following script call. 

    $ python3 /tf/mhcglobe/mhcperf/mhcperf.py {df_train_path} {new_mhcperf_path}

In [11]:
# Training Data for MHCPerf
# Feature column meansings are shown in Supplementary Figure 1.

mhcperf_train_path  = '/tf/fairmhc/mhcglobe/data/mhcperf_data/mhcperf_data.csv'
mhcperf_train_df = pd.read_csv(mhcperf_train_path)
mhcperf_train_df.head()

,allele,fold,trial,left_out_alleles,model_id,PPV,data_aa_pos_1,data_aa_pos_2,data_aa_pos_3,data_aa_pos_4,...,N10_data,dist_bin_0.0,dist_bin_0.1,dist_bin_0.2,dist_bin_0.3,dist_bin_0.4,dist_bin_0.5,dist_bin_0.6,dist_bin_0.7,data_size
0,HLA-B*08:01,1,7,"HLA-B*08:01,HLA-B*08:02,HLA-B*42:02,HLA-B*42:0...",LO_model_37,0.231819,1010678,17258,140937,271779,...,21.0,0.0,0.0,87521.0,100971.0,387642.0,407716.0,26326.0,502.0,1010678
1,HLA-B*08:01,1,9,"HLA-B*08:01,HLA-B*08:02,HLA-B*42:02,HLA-B*42:0...",LO_model_39,0.240697,1006549,17258,136912,267754,...,13.0,0.0,0.0,83392.0,100971.0,387642.0,407716.0,26326.0,502.0,1006549
2,HLA-B*08:01,1,8,"HLA-B*08:01,HLA-B*08:02,HLA-B*42:02,HLA-B*42:0...",LO_model_38,0.251665,1006653,17258,136912,267754,...,1247.0,0.0,0.0,83496.0,100971.0,387642.0,407716.0,26326.0,502.0,1006653
3,HLA-B*08:01,1,4,"HLA-B*08:01,HLA-B*08:02,HLA-B*42:02,HLA-B*42:0...",LO_model_34,0.252187,1013154,17258,143370,274212,...,33460.0,0.0,987.0,89010.0,100971.0,387642.0,407716.0,26326.0,502.0,1013154
4,HLA-B*08:01,1,6,"HLA-B*08:01,HLA-B*08:02,HLA-B*42:02,HLA-B*42:0...",LO_model_36,0.257018,1012167,17258,142426,273268,...,7.0,0.0,0.0,89010.0,100971.0,387642.0,407716.0,26326.0,502.0,1012167


In [8]:
import mhcperf

# Re-train MHCPerf, includes hyperparameter tuning (48 combinations).

# This should be a directory path that connect to the user's local filesystem from Docker.
new_mhcperf_path = '/tf/local/mhcperf_models/mhcperf_example' 

if not os.path.exists(new_mhcperf_path):
    # Train new MHCPerf
    new_mhcperf  = mhcperf.train(
        df_train_path  = mhcperf_train_path,
        model_savepath = new_mhcperf_path,
        verbose        = 0)
else:
    # Load New MHCPerf
    new_mhcperf = mhcperf.model(new_mhcperf_path, mhcperf_train_path)

### 3. Estimate allele-level PPV with new binding data.

Updates an existing mhcperf featurized df to estimate how PPV estimates by MHCPerf
would change if additional data is collected for a given allele of interest. This functionality avoids recomputing the entire feature df for each query allele, and instead updates the existing feature set describing the full training dataset for MHCGlobe.

In [10]:
import featurize_mhcperf

add_n_data = 2000
allele_gets_data = 'HLA-B*15:13'

# Compute updated feature set
df_all_update = featurize_mhcperf.update_mhcperf_features(allele_gets_data, add_n_data)

# Load the fullly trained MHCPerf model.
mhcperf_model = mhcperf.model()

# MHCPerf PPV estimates for all MHC alleles with unique pseudosequence.
ppv_estimates = mhcperf_model.predict_ppv(df_all_update)
df_all_update.insert(1, 'PPV_est', ppv_estimates)
df_all_update[['allele', 'PPV_est']].head()

,allele,PPV_est
0,HLA-A*01:01,0.740534
1,HLA-A*01:02,0.712476
2,HLA-A*01:03,0.705833
3,HLA-A*01:06,0.649255
4,HLA-A*01:07,0.714751
